In [9]:
from plotly.figure_factory import create_dendrogram
import plotly.express as px
import numpy as np
import pandas as pd
import sqlite3

In [10]:
conn = sqlite3.connect('../data_processing/data/nba_shots.db')
df = pd.read_sql('select player, avg_distance, avg_shotX, accuracy, top_quarter from player_profiles', conn)
df.head()

,player,avg_distance,avg_shotX,accuracy,top_quarter
0,A.C. Green,11.464615,26.744615,0.446154,2
1,A.J. Guyton,19.155963,24.989450,0.380734,4
2,A.J. Hammons,15.142857,23.254762,0.404762,4
3,A.J. Price,18.043597,24.121049,0.383515,4
4,Aaron Brooks,14.670170,24.029007,0.417236,4


In [11]:
X = df.drop(columns='player').values
y = df.player.values

In [12]:
from sklearn.preprocessing import StandardScaler
X = StandardScaler().fit_transform(X)

In [23]:
fig2 = create_dendrogram(X, labels=y, color_threshold=100, orientation='left')

target_player = 'Chris Paul'
tick_idx = np.argwhere(fig2.layout.yaxis.ticktext == target_player)[0][0]
# tick_val = fig2.layout.xaxis.tickvals[tick_idx]

zoomed_y_axis = [fig2.layout.yaxis.tickvals[tick_idx - 2], fig2.layout.yaxis.tickvals[tick_idx + 3]]
zoomed_x_axis = [0, 15]

fig2.update_xaxes(range=zoomed_x_axis, minallowed=0)#, rangeslider=dict(visible=True))
fig2.update_yaxes(range=zoomed_y_axis)
fig2.update_layout(autosize=True, margin=dict(t=50, b=50))
fig2.show(config={
    'modeBarButtonsToRemove': ['autoScale', 'resetScale']
})

In [14]:
dim1 = df.avg_distance
dim2 = df.avg_shotX
df['top_quarter'] = df.top_quarter.astype(str)
fig_scatter = px.scatter(x=X[:, 0], y=X[:, 2], hover_name=df.player, hover_data=[df.top_quarter])
fig_scatter.show()
# fig_scatter.data[0].x = dim1[:10]
# fig_scatter.data[0].y = dim1[:10]

In [15]:
# fig_scatter.to_dict()
target_player = 'Chris Paul'
df['target'] = df.player.apply(lambda x: target_player if x == target_player else 'Others') 
df['size'] = df.player.apply(lambda x: 3 if x == target_player else 1)

fig_scatter2 = px.scatter(df, x="avg_distance", y="accuracy", hover_name="player", hover_data="target", color='target', size='size')
tick_idx = np.argwhere(fig_scatter2.data[1].hovertext == target_player)[0][0]
print('Tick idx', tick_idx)

min_x = min(fig_scatter2.data[0].x)
max_x = max(fig_scatter2.data[0].x)
range_x = max_x - min_x

min_y = min(fig_scatter2.data[0].y)
max_y = max(fig_scatter2.data[0].y)
range_y = max_y + min_y

print('range x,y', range_x, range_y)

target_x = fig_scatter2.data[1].x[tick_idx]
target_y = fig_scatter2.data[1].y[tick_idx]

print('target x,y', target_x, target_y)

zoomed_x_axis = [target_x - (0.05 * range_x), target_x + (0.05 * range_x)]
zoomed_y_axis = [target_y - (0.05 * range_y), target_y + (0.05 * range_y)]
print('zoomed_x_range', zoomed_x_axis)
print('zoomed_y_range', zoomed_y_axis)
fig_scatter2.update_xaxes(range=zoomed_x_axis)#, rangeslider=dict(visible=True))
fig_scatter2.update_yaxes(range=zoomed_y_axis)
fig_scatter2.show()

Tick idx 0
range x,y 27.0 1.0
target x,y 14.731657260133401 0.4753434809874009
zoomed_x_range [np.float64(13.381657260133402), np.float64(16.0816572601334)]
zoomed_y_range [np.float64(0.42534348098740093), np.float64(0.5253434809874009)]


In [16]:
from sklearn.decomposition import PCA

pca = PCA()
X_pc = pca.fit_transform(X)
df_pc = pd.DataFrame(X_pc, columns=['PC1', 'PC2', 'PC3', 'PC4'])
df_pc['player'] = df.player
df_pc['target'] = df_pc.player.apply(lambda x: target_player if x == target_player else 'Others') 
df_pc['size'] = df_pc.player.apply(lambda x: 3 if x == target_player else 1)

# df_pc.head()
fig_scatter3 = px.scatter(df_pc, x="PC1", y="PC2", hover_name="player", hover_data="target", color='target', size='size')

min_x = min(fig_scatter3.data[0].x)
max_x = max(fig_scatter3.data[0].x)
range_x = max_x - min_x

min_y = min(fig_scatter3.data[0].y)
max_y = max(fig_scatter3.data[0].y)
range_y = max_y + min_y

tick_idx = np.argwhere(fig_scatter3.data[1].hovertext == target_player)[0][0]
target_x = fig_scatter3.data[1].x[tick_idx]
target_y = fig_scatter3.data[1].y[tick_idx]

zoomed_x_axis = [target_x - (0.05 * range_x), target_x + (0.05 * range_x)]
zoomed_y_axis = [target_y - (0.05 * range_y), target_y + (0.05 * range_y)]
fig_scatter3.update_xaxes(range=zoomed_x_axis)#, rangeslider=dict(visible=True))
fig_scatter3.update_yaxes(range=zoomed_y_axis)
fig_scatter3.show()